# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 1) 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
from collections import Counter

### 2) 데이터 셋 읽어오기

In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [4]:
df_train['business_area'].isnull().sum()

40882

## 2. 피처 클렌징 & 엔지니어링

### 1) 컬럼별


#### customer_country

In [5]:
# 1. customer_country
# country 전처리 함수

def pp_customer_country(text):
    country_map = {
        'u.s.a': ['us', 'usa', 'united states', 'ave', 'dr.', 'dr ', 'drive', 'st', 'ct', 'nevada', 'diablo', 'enterprise dr', 'east fl', 'maxine dr', 'mishawum', 'montour', 'kimball', 'highlands', 'hempston', 'bucaramanga', 'florida', 'hampshire', 'ny', 'ohio', 'roadbostonma', 'ca'],
        'italy': ['italy', 'bari'],
        'u.a.e': ['dubai', 'saudi', 'uae'],
        'antigua and barbuda': ['antigua'],
        'turkey': ['türkiye'],
        'brazil': ['são', 'capão', 'aparecida', 'joão pessoa', 'horizonte', 'cuiabá', 'dourados', 'manaus', 'recife', 'janeiro', 'rj', 'paulo'],
        'spain': ['madrid', 'elche', 'agost', 'caceres', 'canarias', 'valencia'],
        'netherlands': ['netherlands', 'curaçao', 'aruba', 'maarten'],
        'vietnam': ['ha noi'],
        'congo': ['congo'],
        'colombia': ['colombia', 'barranquilla', 'carrera', 'cartagena'],
        'india': ['indore', 'lucknow', 'chennai', 'anand', 'bangalore', 'bhilwara', 'gujarat', 'gurgaon', 'hyderabad', 'kerela', 'mumbai', 'odisha', 'pune', 'telangana', 'pradesh'],
        'greece': ['θ'],
        'nigeria': ['benin'],
        'saint kitts and nevis': ['st kitts'],
        'nan': ['', 'nd', 'br', '5555', 'a']
    }

    text = str(text).lower().split('/')[-1].strip()

    for country, keys in country_map.items():
        for k in keys:
            if k in text:
                return country
        
    if '@' in text:
        return 'email'

    if re.search(r'\d{6,}', text) or re.search(r'\d{5,}', text):
        return 'u.s.a'

#     return text

# train, test셋에 전처리 함수 적용
df_train['customer_country'] = df_train['customer_country'].apply(pp_customer_country)
df_test['customer_country'] = df_test['customer_country'].apply(pp_customer_country)

#### customer_type, customer_job

- 수정 버전

In [6]:
def split_word(temp):
    if isinstance(temp, str):  # 문자열인지 확인
        temp = [temp]  # 문자열을 리스트로 변환하여 단일 값도 처리 가능하도록 함
    result = []
    for item in temp:
        if isinstance(item, str):  # 문자열인지 다시 확인
            if "/" in item:
                sen_temp = item.split("/")
            elif "&" in item:
                sen_temp = item.split("&")
            elif "_" in item:
                sen_temp = item.split("_")
            elif "," in item:
                sen_temp = item.split(",")
            elif "." in item:
                sen_temp = item.split(".")
            else:
                sen_temp = [item]  # 위 조건에 해당되지 않으면 그대로 유지
            sen_temp = [word.strip() for word in sen_temp]
            result.append(" ".join(sen_temp))
        else:
            result.append(item)  # 문자열이 아닌 경우 그대로 유지
    return result


In [7]:
df_train.loc[:,'customer_job']= split_word(df_train['customer_job'])

In [8]:
df_test.loc[:,'customer_job']= split_word(df_test['customer_job'])

In [9]:
def pp_customer_job(row):
    customer_jobs = row['customer_job']
    
    if isinstance(customer_jobs, str):  # 문자열인지 확인
        categories = ['art and design', 'marketing', 'education', 'information technology', 'administrative', 'sales', 'operation', 'program and project management', 'film production', 'finance', 'human resources', 'accounting', 'media and communication', 'legal', 'biomedical', 'construction', 'property', 'R&D','other']
        display_keywords = {
            'art and design': ['arts', 'art', 'arts and design', 'arte y diseño', 'design', 'artist, lead on equipment selection', 'arte_e_design', 'művészet_és_design'],
            'marketing': ['marketing', 'advertising'],
            'education': ['educator', 'education', 'teacher'],
            'information technology': ['engineer', 'develop', 'si', 'it', 'technology', 'chief eng'],
            'administrative': ['administration', 'administración', 'admin', 'administrative'],
            'sales' : ['sales', 'sale', 'vendite', 'értékesítés', 'salesman'],
            'operation': ['operations executive', 'regional director of operations', 'operations manager', 'director of operations', 'strategy & operations specialist', 'facilities and operations', 'operaciones'],
            'program and project management': ['project', 'genel müdür', 'gestión_de_proyectos', 'programm'],
            'film production': ['film production'],
            'finance': ['finance','finanzas', 'finanzen', 'pénzügy', 'vertrieb'],
            'human resources': ['hr','human resources', 'human resource'],
            'accounting': ['account', 'accounting', 'account management', 'account manager'],
            'media and communication' : ['media', 'media e comunicazione', 'média és kommunikáció', 'medios de comunicación', 'medien und kommunikation', 'communication'],
            'legal' : ['legal'],
            'biomedical' : ['medical solution', 'healthcare', 'spécialiste_en_imagerie_médicale', 'medical imaging'], 
            'medical' : ['doctor', 'nurse', 'pathologist', 'profesional de cirugía', 'surgery professional\u200b', 'radiology professional', 'főorvos', 'clinic', 'radiology_professional', 'profesional de radiología', 'cirugano', 'chirurgien', 'surgery professional'],
            'construction' : ['architect', 'project architect', 'contractor' ],
            'property' : ['real estate', 'proprietário(a)'],
            'R&D': ['research', 'R&D', 'r&d', ],
            'other': ['other', 'others', 'otherss']
        }

        for category in categories:
            for keyword in display_keywords[category]:
                if keyword in customer_jobs:
                    return category

    return 'other'


In [10]:
df_train['customer_job'] = df_train.apply(pp_customer_job, axis=1)

In [11]:
df_test['customer_job'] = df_test.apply(pp_customer_job, axis=1)

- 이전 버전

In [12]:
# # customer_type & job
# # train 셋

# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('End-Customer','End Customer'))
# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('End-user','End Customer'))
# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('Commercial end-user','End Customer'))
# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('Home Owner','Homeowner'))
# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('Other','others'))
# df_train['customer_type'] = df_train['customer_type'].apply(lambda x: str(x).replace('Etc.','others'))

# df_train.loc[(df_train['customer_type']=='nan') & (df_train['customer_job'] == 'nan') & (df_train['customer_position'] == 'none'), 'customer_job'] = 'Empthy'
# df_train.loc[(df_train['customer_type']=='nan') & (df_train['customer_job'] == 'nan') & (df_train['customer_position'] != 'none'), ['customer_job','customer_type']] = 'others'
# df_train.loc[df_train['customer_job'] == 'egyéb', 'customer_job'] = 'others'
# df_train.loc[(df_train['customer_type']== 'nan') & (df_train['customer_job'] == 'others'), ['customer_job','customer_type']]= 'others'
# df_train.loc[(df_train['customer_type']== 'others') & (df_train['customer_job'] == 'nan'), ['customer_job','customer_type']]= 'others'
# df_train.loc[(df_train['customer_job'] == 'otherss') | (df_train['customer_job'] ==  'sonstiges') | (df_train['customer_job'] == 'otro') |(df_train['customer_job'] == 'autres'), 'customer_job']= 'others'

# df_train.loc[df_train['customer_job'] == 'engineer', 'customer_job']= 'engineering'
# df_train.loc[(df_train['customer_type'] == 'Engineer') | (df_train['customer_type'] == 'HVAC Engineer'), 'customer_job']= 'engineering'
# df_train.loc[(df_train['customer_job'] == 'finanzen') | (df_train['customer_job'] == 'pénzügy')  | (df_train['customer_job'] == 'finanzas') | (df_train['customer_job'] ==  'finance executive'), 'customer_job']= 'finance'
# df_train.loc[(df_train['customer_job'] == 'product marketing') | (df_train['customer_job'] == 'technical marketing'), 'customer_job']= 'marketing'
# df_train.loc[(df_train['customer_job'] == 'event marketing') | (df_train['customer_job'] == 'field marketing') | (df_train['customer_job'] == 'advertising'), 'customer_job']= 'marketing'

# df_train.loc[(df_train['customer_job'] == 'vendite') | (df_train['customer_job'] == 'értékesítés') | (df_train['customer_job'] == 'sales engineering') |(df_train['customer_job'] == 'vertrieb'), 'customer_job']= 'sales'

# df_train.loc[(df_train['customer_job'] == 'projektmenedzsment\tprogram and project management') | (df_train['customer_job'] == 'program-_és_projektmenedzsment') | (df_train['customer_job'] == 'programm-_und_projektmanagement') | (df_train['customer_job'] == 'project manager') | (df_train['customer_job'] == 'program_and_project_manager')| (df_train['customer_job'] == 'programm- und projektmanagement'), 'customer_job']= 'program and project management'
# df_train.loc[(df_train['customer_job'] == 'designere / budget') | (df_train['customer_job'] == 'genel müdür')| (df_train['customer_job'] == 'program_and_project_management') | (df_train['customer_job'] == 'gestión_de_proyectos'), 'customer_job']= 'program and project management'

# df_train.loc[(df_train['customer_job'] == 'public bidder') | (df_train['customer_job'] == 'pénzügy'), 'customer_job']= 'bidder'

# df_train.loc[df_train['customer_job'] == 'ranger 2', 'customer_job'] ='ranger'

# df_train.loc[(df_train['customer_job']== 'nan') & (df_train['customer_position']== 'none')& (df_train['customer_type']== 'Consultant'), 'customer_job'] = 'consulting'
# df_train.loc[(df_train['customer_job'] == 'consultant') | (df_train['customer_job'] == 'consultent') | (df_train['customer_job'] == 'consultant / purchaser')| (df_train['customer_job'] == 'recommend (you recommend specific products or technologies for the solution)'), 'customer_job']= 'consulting'
# df_train.loc[(df_train['customer_job']== 'nan') & (df_train['business_subarea']== 'Consulting') , 'customer_job'] = 'consulting'

# df_train.loc[(df_train['customer_job']== 'nan') & (df_train['customer_type']== 'Specifier/ Influencer') , 'customer_job'] = 'influencer'

# df_train.loc[(df_train['customer_job']== 'nan') & (df_train['customer_type']== 'End Customer')  & (df_train['customer_position']== 'none') , 'customer_job']='Empthy'
# df_train.loc[(df_train['customer_job']== 'nan') & (df_train['customer_type']== 'Channel Partner')  , 'customer_job']= 'Channel Partner'

# df_train.loc[(df_train['customer_job'] ==  'művészet_és_design') |(df_train['customer_job'] ==  'arte_e_design') |(df_train['customer_job'] == 'artist, lead on equipment selection') |(df_train['customer_job'] == 'design') |(df_train['customer_job'] == 'arts_and_design') |(df_train['customer_job'] == 'arte y diseño') |(df_train['customer_job'] == 'arts and design'), 'customer_job'] = 'art and design'
# df_train.loc[(df_train['customer_job'] ==  'graphic/color art') |(df_train['customer_job'] ==  'designer') |(df_train['customer_job'] ==  'colorist') |(df_train['customer_job'] ==  'designer, producer') |(df_train['customer_job'] ==  'lead designer') |(df_train['customer_job'] ==  'designers'), 'customer_job']= 'art and design'  
             
# df_train.loc[(df_train['customer_job'] == 'media_e_comunicazione') |(df_train['customer_job'] ==  'média_és_kommunikáció') |(df_train['customer_job'] ==  'medios_de_comunicación') |(df_train['customer_job'] == 'medien_und_kommunikation') |(df_train['customer_job'] == 'media_and_communication'), 'customer_job'] = 'media and communication'

# df_train.loc[(df_train['customer_job'] == 'educator') | (df_train['customer_job'] ==  'higher education (college & university)'), 'customer_job'] = 'education'

# df_train.loc[(df_train['customer_job'] == 'adminisztráció') |(df_train['customer_job'] ==   'amministrativo') |(df_train['customer_job'] ==  'administration') |(df_train['customer_job'] ==  'administración'), 'customer_job'] =  'administrative'

# df_train.loc[(df_train['customer_job'] == 'systems designer') |(df_train['customer_job'] == 'information_technology') |(df_train['customer_job'] == 'computing & it') |(df_train['customer_job'] == 'it/software') |(df_train['customer_job'] == 'it')|(df_train['customer_job'] == 'information technology\u200b')|(df_train['customer_job'] == 'graphic design')|(df_train['customer_job'] == 'it - information technology') |(df_train['customer_job'] == 'collaboration & web apps') |(df_train['customer_job'] == 'it integrator'), 'customer_job'] =  'information technology'
# df_train.loc[(df_train['customer_job'] ==  'application development') |(df_train['customer_job'] == 'si') |(df_train['customer_job'] == 'system engineer') |(df_train['customer_job'] == 'developer') |(df_train['customer_job'] == 'design engineer') |(df_train['customer_job'] == 'it dairector') |(df_train['customer_job'] == 'it department') |(df_train['customer_job'] == 'it hardware technician'), 'customer_job'] =  'information technology'
# df_train.loc[(df_train['customer_job'] =='director of it') | (df_train['customer_job'] == 'it director')  | (df_train['customer_job'] == 'it manager') | (df_train['customer_job'] ==  'it specialist') |(df_train['customer_job'] == 'systems design') |(df_train['customer_job'] == 'it tech.') |(df_train['customer_job'] == 'it admin') |(df_train['customer_job'] == 'engineering, design, and install') |(df_train['customer_job'] == 'system designer, integrator'), 'customer_job']= 'information technology'

# df_train.loc[(df_train['customer_job'] ==  'account exec/manager') |(df_train['customer_job'] ==   'account management') |(df_train['customer_job'] ==  'accounts payable'), 'customer_job'] =  'accounting'

# df_train.loc[(df_train['customer_job'] ==  'medical imaging  specialist') |(df_train['customer_job'] ==   'medical solution provider') |(df_train['customer_job'] ==   'medical solution provider\u200b') |(df_train['customer_job'] ==   'healthcare professionals') |(df_train['customer_job'] ==   'healthcare services') |(df_train['customer_job'] ==   'spécialiste_en_imagerie_médicale') |(df_train['customer_job'] ==   'healthcare_services'), 'customer_job'] =  'biomedical'
# df_train.loc[(df_train['customer_job'] ==  'profesional de cirugía') |(df_train['customer_job'] ==   'surgery professional\u200b') |(df_train['customer_job'] ==    'radiology professional') |(df_train['customer_job'] ==   'főorvos') |(df_train['customer_job'] ==   'clinic')|(df_train['customer_job'] ==  'radiology_professional') |(df_train['customer_job'] ==  'profesional de radiología') |(df_train['customer_job'] ==  'cirugano') |(df_train['customer_job'] ==   'chirurgien') |(df_train['customer_job'] ==   'surgery professional'), 'customer_job'] =  'medical'

# df_train.loc[(df_train['customer_job'] == 'üzemeltetés') |(df_train['customer_job'] == 'operations executive') |(df_train['customer_job'] ==  'regional director of operations') |(df_train['customer_job'] ==  'operations manager') |(df_train['customer_job'] == 'director of operations')|(df_train['customer_job'] == 'strategy & operations specialist')|(df_train['customer_job'] ==  'facilities and operations')|(df_train['customer_job'] ==  'operaciones'), 'customer_job'] =  'operations'

# df_train.loc[(df_train['customer_job'] == 'real estate') | (df_train['customer_job'] == 'proprietário(a)'), 'customer_job'] = 'property'

In [13]:
# # test 셋

# # customer_type 전처리
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('End-Customer','End Customer'))
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('End-user','End Customer'))
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('Commercial end-user','End Customer'))
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('Home Owner','Homeowner'))
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('Other','others'))
# df_test['customer_type'] = df_test['customer_type'].apply(lambda x: str(x).replace('Etc.','others'))

# # customer_job 전처리
# df_test.loc[(df_test['customer_type']=='nan') & (df_test['customer_job'] == 'nan') & (df_test['customer_position'] == 'none'), 'customer_job'] = 'Empthy'
# df_test.loc[(df_test['customer_type']=='nan') & (df_test['customer_job'] == 'nan') & (df_test['customer_position'] != 'none'), ['customer_job','customer_type']] = 'others'
# df_test.loc[df_test['customer_job'] == 'egyéb', 'customer_job'] = 'others'
# df_test.loc[(df_test['customer_type']== 'nan') & (df_test['customer_job'] == 'others'), ['customer_job','customer_type']]= 'others'
# df_test.loc[(df_test['customer_type']== 'others') & (df_test['customer_job'] == 'nan'), ['customer_job','customer_type']]= 'others'
# df_test.loc[(df_test['customer_job'] == 'otherss') | (df_test['customer_job'] ==  'sonstiges') | (df_test['customer_job'] == 'otro') |(df_test['customer_job'] == 'autres'), 'customer_job']= 'others'
# df_test.loc[df_test['customer_job'] == 'engineer', 'customer_job']= 'engineering'
# df_test.loc[(df_test['customer_type'] == 'Engineer') | (df_test['customer_type'] == 'HVAC Engineer'), 'customer_job']= 'engineering'
# df_test.loc[(df_test['customer_job'] == 'finanzen') | (df_test['customer_job'] == 'pénzügy')  | (df_test['customer_job'] == 'finanzas') | (df_test['customer_job'] ==  'finance executive'), 'customer_job']= 'finance'
# df_test.loc[(df_test['customer_job'] == 'product marketing') | (df_test['customer_job'] == 'technical marketing'), 'customer_job']= 'marketing'
# df_test.loc[(df_test['customer_job'] == 'event marketing') | (df_test['customer_job'] == 'field marketing') | (df_test['customer_job'] == 'advertising'), 'customer_job']= 'marketing'
# df_test.loc[(df_test['customer_job'] == 'vendite') | (df_test['customer_job'] == 'értékesítés') | (df_test['customer_job'] == 'sales engineering') |(df_test['customer_job'] == 'vertrieb'), 'customer_job']= 'sales'
# df_test.loc[(df_test['customer_job'] == 'projektmenedzsment\tprogram and project management') | (df_test['customer_job'] == 'program-_és_projektmenedzsment') | (df_test['customer_job'] == 'programm-_und_projektmanagement') | (df_test['customer_job'] == 'project manager') | (df_test['customer_job'] == 'program_and_project_manager')| (df_test['customer_job'] == 'programm- und projektmanagement'), 'customer_job']= 'program and project management'
# df_test.loc[(df_test['customer_job'] == 'designere / budget') | (df_test['customer_job'] == 'genel müdür')| (df_test['customer_job'] == 'program_and_project_management') | (df_test['customer_job'] == 'gestión_de_proyectos'), 'customer_job']= 'program and project management'
# df_test.loc[(df_test['customer_job'] == 'public bidder') | (df_test['customer_job'] == 'pénzügy'), 'customer_job']= 'bidder'
# df_test.loc[df_test['customer_job'] == 'ranger 2', 'customer_job'] ='ranger'
# df_test.loc[(df_test['customer_job']== 'nan') & (df_test['customer_position']== 'none')& (df_test['customer_type']== 'Consultant'), 'customer_job'] = 'consulting'
# df_test.loc[(df_test['customer_job'] == 'consultant') | (df_test['customer_job'] == 'consultent') | (df_test['customer_job'] == 'consultant / purchaser')| (df_test['customer_job'] == 'recommend (you recommend specific products or technologies for the solution)'), 'customer_job']= 'consulting'
# df_test.loc[(df_test['customer_job']== 'nan') & (df_test['business_subarea']== 'Consulting') , 'customer_job'] = 'consulting'
# df_test.loc[(df_test['customer_job']== 'nan') & (df_test['customer_type']== 'Specifier/ Influencer') , 'customer_job'] = 'influencer'
# df_test.loc[(df_test['customer_job']== 'nan') & (df_test['customer_type']== 'End Customer')  & (df_test['customer_position']== 'none') , 'customer_job']='Empthy'
# df_test.loc[(df_test['customer_job']== 'nan') & (df_test['customer_type']== 'Channel Partner')  , 'customer_job']= 'Channel Partner'

# df_test.loc[(df_test['customer_job'] == 'művészet_és_design') | (df_test['customer_job'] == 'arte_e_design') | (df_test['customer_job'] == 'artist, lead on equipment selection') | (df_test['customer_job'] == 'design') | (df_test['customer_job'] == 'arts_and_design') | (df_test['customer_job'] == 'arte y diseño') | (df_test['customer_job'] == 'arts and design'), 'customer_job'] = 'art and design'
# df_test.loc[(df_test['customer_job'] == 'graphic/color art') | (df_test['customer_job'] == 'designer') | (df_test['customer_job'] == 'colorist') | (df_test['customer_job'] == 'designer, producer') | (df_test['customer_job'] == 'lead designer') | (df_test['customer_job'] == 'designers'), 'customer_job'] = 'art and design'

# df_test.loc[(df_test['customer_job'] == 'media_e_comunicazione') | (df_test['customer_job'] == 'média_és_kommunikáció') | (df_test['customer_job'] == 'medios_de_comunicación') | (df_test['customer_job'] == 'medien_und_kommunikation') | (df_test['customer_job'] == 'media_and_communication'), 'customer_job'] = 'media and communication'

# df_test.loc[(df_test['customer_job'] == 'educator') | (df_test['customer_job'] == 'higher education (college & university)'), 'customer_job'] = 'education'

# df_test.loc[(df_test['customer_job'] == 'adminisztráció') | (df_test['customer_job'] == 'amministrativo') | (df_test['customer_job'] == 'administration') | (df_test['customer_job'] == 'administración'), 'customer_job'] = 'administrative'

# df_test.loc[(df_test['customer_job'] == 'systems designer') | (df_test['customer_job'] == 'information_technology') | (df_test['customer_job'] == 'computing & it') | (df_test['customer_job'] == 'it/software') | (df_test['customer_job'] == 'it') | (df_test['customer_job'] == 'information technology\u200b') | (df_test['customer_job'] == 'graphic design') | (df_test['customer_job'] == 'it - information technology') | (df_test['customer_job'] == 'collaboration & web apps') | (df_test['customer_job'] == 'it integrator'), 'customer_job'] = 'information technology'
# df_test.loc[(df_test['customer_job'] == 'application development') | (df_test['customer_job'] == 'si') | (df_test['customer_job'] == 'system engineer') | (df_test['customer_job'] == 'developer') | (df_test['customer_job'] == 'design engineer') | (df_test['customer_job'] == 'it dairector') | (df_test['customer_job'] == 'it department') | (df_test['customer_job'] == 'it hardware technician'), 'customer_job'] = 'information technology'
# df_test.loc[(df_test['customer_job'] == 'director of it') | (df_test['customer_job'] == 'it director') | (df_test['customer_job'] == 'it manager') | (df_test['customer_job'] == 'it specialist') | (df_test['customer_job'] == 'systems design') | (df_test['customer_job'] == 'it tech.') | (df_test['customer_job'] == 'it admin') | (df_test['customer_job'] == 'engineering, design, and install') | (df_test['customer_job'] == 'system designer, integrator'), 'customer_job'] = 'information technology'

# df_test.loc[(df_test['customer_job'] == 'account exec/manager') | (df_test['customer_job'] == 'account management') | (df_test['customer_job'] == 'accounts payable'), 'customer_job'] = 'accounting'

# df_test.loc[(df_test['customer_job'] == 'medical imaging  specialist') | (df_test['customer_job'] == 'medical solution provider') | (df_test['customer_job'] == 'medical solution provider\u200b') | (df_test['customer_job'] == 'healthcare professionals') | (df_test['customer_job'] == 'healthcare services') | (df_test['customer_job'] == 'spécialiste_en_imagerie_médicale') | (df_test['customer_job'] == 'healthcare_services'), 'customer_job'] = 'biomedical'
# df_test.loc[(df_test['customer_job'] == 'profesional de cirugía') | (df_test['customer_job'] == 'surgery professional\u200b') | (df_test['customer_job'] == 'radiology professional') | (df_test['customer_job'] == 'főorvos') | (df_test['customer_job'] == 'clinic') | (df_test['customer_job'] == 'radiology_professional') | (df_test['customer_job'] == 'profesional de radiología') | (df_test['customer_job'] == 'cirugano') | (df_test['customer_job'] == 'chirurgien') | (df_test['customer_job'] == 'surgery professional'), 'customer_job'] = 'medical'

# # customer_job 전처리
# df_test.loc[(df_test['customer_job'] == 'üzemeltetés') | (df_test['customer_job'] == 'operations executive') | (df_test['customer_job'] == 'regional director of operations') | (df_test['customer_job'] == 'operations manager') | (df_test['customer_job'] == 'director of operations') | (df_test['customer_job'] == 'strategy & operations specialist') | (df_test['customer_job'] == 'facilities and operations') | (df_test['customer_job'] == 'operaciones'), 'customer_job'] = 'operations'

# df_test.loc[(df_test['customer_job'] == 'real estate') | (df_test['customer_job'] == 'proprietário(a)'), 'customer_job'] = 'property'


#### historical_existing_cnt

In [14]:
df_train['historical_existing_cnt'] = df_train['historical_existing_cnt'].fillna(0)
# df_train = df_train[df_train['historical_existing_cnt']<1000]

####  inquiry_type

- train set

In [15]:
def preprocess_inquiry_type_1(df):
    # inquiry_type 열을 소문자로 변환
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    
    # 대체할 문자열과 대상 문자열 정의
    replacement_mapping = {
        '(select id_needs)':'etc',
        'customer suggestions':'customer suggestions',
        'digital platform':'product information',
        'display product':'product information',
        'display textbook and photos':'product information',
        'educational equipments':'product information',
        'estoy buscando para ecuador este producto lg magnit micro led, para un cliente de 138 pulgadas, con envió marítimo.':'product purchase inquiry',
        'etc.':'etc',
        'event inquiry':'etc',
        'evento_sdelestero':'etc',
        'first info and pricing':'pricing and demo request',
        'for school':'product information',
        'high inch 86 / 98 or 110':'product information',
        'hola me pueden cotizar 19 pantallas interactivas de 100 pulgadas entregadas en guayaquil -ecuador.':'quotation and purchase Inquiry',
        'hospital tv':'product information',
        'hotel tv products':'product information',
        'i want to know the details about it':'product information',
        'idb':'product and technical support/consultation',
        'intégrateur historique du george v':'intégrateur historique du george v',
        'led signage':'product information',
        'media inquiry':'customer suggestions',
        'needs':'etc',
        'not specified':'other',
        'oem/odm request':'product and technical support/consultation',
        'one quick:flex':'product information',
        'other_':'other',
        'others':'other',
        'pantallas interactivas para clinicas':'product information',
        'preciso de um monitor médico para radiografia convencional e tomogrtafia.':'product information',
        'probeam precio':'pricing and demo request',
        'purchase or quotation':'purchase',
        'quotation or purchase consultation':'purchase',
        'quotation_':'purchase',
        'quotation_or_purchase_consultation':'purchase',
        'request a demo':'pricing and demo request',
        'request for distributorship':'partnership/distributorship request',
        'request for partnership':'partnership/distributorship request',
        'request for quotation or purchase':'quotation and purchase Inquiry',
        'request for technical consulting':'product and technical support/consultation',
        'sales inquiry':'sales',
        'services':'product and technical support/consultation',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución one quick: ':'quotation and purchase Inquiry',
        'standalone':'product information',
        'teach':'trainings',
        'technical consultation':'technical',
        'technical support':'technical',
        'technical_consultation':'technical',
        'toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung':'quotation and purchase Inquiry',
        'tv interactive':'product information',
        'tôi cần tham khảo giá và giải pháp từ lg':'quotation and purchase Inquiry',
        'usage or technical consultation':'product and technical support/consultation',
        'usage_or_technical_consultation':'product and technical support/consultation',
        'video wall':'product and technical support/consultation',
        'vrf':'product information',
        'vui lòng báo giá giúp mình sản phẩm đo thân nhiệt xin cảm ơn':'product information',
        'window facing product':'product information',
        'aio':'product information',
        'product purchase inquiry':'purchase',
        'purchaseor_purchase_consultation':'purchase',
        'quotation and purchase inquiry':'purchase',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución one quick: ':'purchase',
        'usage or technical':'technical',
        'usage_or_technical':'technical',
        'customer suggestions': 'etc',
        'intégrateur historique du george v': 'etc',
        'solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la solución\xa0one quick:\xa0': 'purchase',
        'product and technical support/consultation':'product and technical consultation',
        'product and technical/consultation':'product and technical consultation',
        'product and technical consultation':'technical',
        'technical':'product and technical consultation',
        'quotation and purchase inquiry':'purchase',
        'purchase':'quotation and purchase inquiry',
        'product and product and product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'product and product and product and product and product and technical consultation':'product and technical consultation',
        'product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and purchase inquiryor_quotation and purchase inquiry_consultation':'quotation and purchase inquiry',
        'quotation or quotation and purchase inquiry consultation':'quotation and purchase inquiry',
        'request for product and product and product and product and product and technical consultation consulting':'product and technical consultation',
        'request for quotation or quotation and purchase inquiry':'quotation and purchase inquiry',
        'usage or product and product and product and product and product and technical consultation':'product and technical consultation',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
        'product and product and product and technical consultation':'product and technical consultation',
        'customer suggestions':'etc',
        'quotation and quotation and purchase inquiry inquiry':'product and technical consultation',
    }

    # 대상 문자열에 대해 대체 수행
    for old_str, new_str in replacement_mapping.items():
        # 괄호가 있는 경우
        if '(' in old_str:
            df['inquiry_type'] = df['inquiry_type'].replace(r'\('+old_str.strip('()')+r'\)', new_str, regex=True)
        # 괄호가 없는 경우
        else:
            df['inquiry_type'] = df['inquiry_type'].replace(old_str, new_str, regex=True)

    # inquiry_type 열의 값을 소문자로 변환하여 Counter에 전달
    inquiry_type_counts = Counter(str(value).lower() for value in df['inquiry_type'] if not isinstance(value, float))

    # 총 고유값 개수 확인
    total_unique_values = len(inquiry_type_counts)
    print("총 고유값 개수:", total_unique_values)

    # 각 고유값과 빈도수 출력
    print("고유값 및 빈도수:")
    for value, count in sorted(inquiry_type_counts.items()):
        print(f"'{value}':'{count}'")


    # 최빈값 계산
    mode_value = df['inquiry_type'].mode()[0]

    # 결측치를 최빈값으로 대체
    df['inquiry_type'].fillna(mode_value, inplace=True)
    print('---------------------')
    return df

In [16]:
def preprocess_inquiry_type_2(df):
    # inquiry_type 열을 소문자로 변환
    df['inquiry_type'] = df['inquiry_type'].str.lower()
    
    # 대체할 문자열과 대상 문자열 정의
    replacement_mapping = {
        'customer suggestions':'etc',
        'quotation and quotation and purchase inquiry inquiry':'quotation and purchase inquiry',
    }

    # 대상 문자열에 대해 대체 수행
    for old_str, new_str in replacement_mapping.items():
        # 괄호가 있는 경우
        if '(' in old_str:
            df['inquiry_type'] = df['inquiry_type'].replace(r'\('+old_str.strip('()')+r'\)', new_str, regex=True)
        # 괄호가 없는 경우
        else:
            df['inquiry_type'] = df['inquiry_type'].replace(old_str, new_str, regex=True)

    # inquiry_type 열의 값을 소문자로 변환하여 Counter에 전달
    inquiry_type_counts = Counter(str(value).lower() for value in df['inquiry_type'] if not isinstance(value, float))

    # 총 고유값 개수 확인
    total_unique_values = len(inquiry_type_counts)
    print("총 고유값 개수:", total_unique_values)
    

    # 각 고유값과 빈도수 출력
    print("고유값 및 빈도수:")
    for value, count in sorted(inquiry_type_counts.items()):
        print(f"'{value}':'{count}'")

    # 최빈값 계산
    mode_value = df['inquiry_type'].mode()[0]

    # 결측치를 최빈값으로 대체
    df['inquiry_type'].fillna(mode_value, inplace=True)
    print('---------------------')
    return df


In [17]:
df_train = preprocess_inquiry_type_1(df_train)
df_train = preprocess_inquiry_type_2(df_train)

df_test = preprocess_inquiry_type_1(df_test)
df_test = preprocess_inquiry_type_2(df_test)

총 고유값 개수: 11
고유값 및 빈도수:
'customer suggestions':'2'
'etc':'242'
'other':'1181'
'partnership/distributorship request':'372'
'pricing and demo request':'187'
'product and technical consultation':'2210'
'product information':'1257'
'quotation and purchase inquiry':'42156'
'quotation and quotation and purchase inquiry inquiry':'235'
'sales':'10081'
'trainings':'435'
---------------------
총 고유값 개수: 9
고유값 및 빈도수:
'etc':'244'
'other':'1181'
'partnership/distributorship request':'372'
'pricing and demo request':'187'
'product and technical consultation':'2210'
'product information':'1257'
'quotation and purchase inquiry':'43332'
'sales':'10081'
'trainings':'435'
---------------------
총 고유값 개수: 9
고유값 및 빈도수:
'customer suggestions':'2'
'etc':'4'
'other':'122'
'partnership/distributorship request':'64'
'pricing and demo request':'9'
'product and technical consultation':'224'
'product information':'68'
'quotation and purchase inquiry':'3465'
'trainings':'21'
---------------------
총 고유값 개수: 8
고유값 및 빈도

#### product_category

In [18]:
def pp_pc(row):
#     categories = ['display', 'tv', 'temperature', 'air', 'beam', 'beauty', 'board', 'cloud', 'pc', 'software', 'refrigerator', 'robot', 'energy', 'other']
    category_dict = {
        'display': ['Display','video', 'signage', 'Signage', '43uh5f', '49uh', '49vl', '49vl', '55vm', '86uh', '98uh', 'display', 'uh5f', 'fhd', 'gsca',
                    'gscd', 'brightness', 'laec', 'leadallin', 'bloc', 'one:quick', 'lsca', 'screen', '顯示屏',
                    'one quick', 'onequick', 'led', 'lcd', 'oled', 'ultra stretch series', 'ur640', 'videowall', 'videwall', 'Monitors',
                    'virtual production', 'window facing', 'pol', '28mq780', 'monitor', 'UM3DG', 'UN880', 'MQ780', 'QP88D', 'ERGO', 'UH', '65EP5G',
                    'UL3J', 'WN780', 'svh7f', 'uh', '32SM5J', 'inch', 'essential series'],
        'tv': ['43uq751c0sb', 'lq621cbsb', '43us660h', '50uq801', '50us660', 'tv', 'TV', 'us670', 'pro centric', 'procentric', 'smart tv', '電視', 'CT5WJ', 'HT3WJ', 'uq801c0sb', 'uq751c0sf',
               'us660h0sd'],
        'temperature': ['definir', 'thermodynamic water heater', 'ac rumah', 'acondicionado', 'condicionado' 'vrf', 'aquecimento',
                        'calefacción', 'chiller', 'climatiseur', 'heating', 'isıtma', 'split', 'magnit', 'inverter', 'multi', 'aio',
                        'ogrzewanie', 'pendingin', 'rac', 'cac', 'conditioner', 'sac', 'scroll compressor', 'soğutucu',
                        'system ac', 'réfrigérant', 'cassete inverter', 'điều hòa', 'אחר', 'חימום', 'מזגנים ', 'מרובה ', 'تكييف وتبريد', 'تكييفات',
                        'เครื่องปรับอากาศเผื่อที่อยู่อาศัย', 'مبرد', 'reversible ac', 'single package', 'حلول التدفئة', 'פיצול מרובה', 'unitario'],
        'air': ['ahu', 'cac', 'air solution', 'air solution', 'ventilation', 'aircare', 'vrf'],
        'beam': ['bu50nst', 'projector', 'Beam', 'Projector'],
        'beauty' : ['beauty'],
        'board': ['createboard', 'idb', 'borad', 'tr3', '55tc3d'],
        'cloud': ['cloud', 'Cloud', 'id', 'cloud device', 'Thin Clients', 'Zero Clients'],
        'pc': ['laptop', 'pc', 'Laptops'],
        'software': ['pro centric', 'procentric', 'software', 'software solution', 'webos', '軟體', 'pro:centric', 'SuperSign'],
        'refrigerator': ['refrigerator'],
        'robot': ['robot'],
        'energy': ['solar', 'energy', 'energy storage system'],
        'other': ['autre', 'inne', 'khác', 'etc.', 'lainnya', 'not specified', 'other', 'Other', 'otros', 'outros', 'ฯลฯ', '其他'],
        'parts_and_accessories' : ['parts', 'accessories', 'Accessories', 'Antennas'],
        'plug' : ['SC-00DA'],
        'appliances' : ['washing machine', 'dryer', 'vb.', 'vacuum'],
        'hospital' : ['Medical', 'medical', 'hospital', 'X-ray', 'HN713D', 'HQ513D', 'surgical', 'จอภาพเพื่อการวินิจฉัย', 'จอภาพสำหรับการตรวจสอบทางคลินิก'],
        'services' : ['support', 'care program', 'Enqiry', 'inquiry', 'service', 'error']
    }
    
    for category, keywords in category_dict.items():
        for keyword in keywords:
            if keyword in row['product_category']:
                row[category] = 1
                break
                
#           
#     for category in categories:
#         for keyword in keywords[category]:
#             if keyword in row['product_category']:
#                 row[category] = 1
#                 break

    return row


In [19]:
product_train = df_train[['product_category', 'product_subcategory', 'product_modelname']]
product_train = product_train.assign(com=product_train.apply(lambda row: ''.join(str(cell).replace('nan', '') for cell in row), axis=1))

category_table_train = pd.DataFrame(product_train['com'])
category_table_train.rename(columns={'com': 'product_category'}, inplace=True)
category_table_train[['display', 'tv', 'temperature', 'air', 'beam', 'beauty', 'board', 'cloud', 'pc', 'software', 'refrigerator', 'robot', 'energy', 'other', 'parts_and_accessories', 'plug', 'appliances', 'hospital', 'services']] = 0
category_table_train = category_table_train.fillna('None')
category_table_train = category_table_train.apply(pp_pc, axis=1)


product_test = df_test[['product_category', 'product_subcategory', 'product_modelname']]
product_test = product_test.assign(com=product_test.apply(lambda row: ''.join(str(cell).replace('nan', '') for cell in row), axis=1))

category_table_test = pd.DataFrame(product_test['com'])
category_table_test.rename(columns={'com': 'product_category'}, inplace=True)
category_table_test[['display', 'tv', 'temperature', 'air', 'beam', 'beauty', 'board', 'cloud', 'pc', 'software', 'refrigerator', 'robot', 'energy', 'other', 'parts_and_accessories', 'plug', 'appliances', 'hospital', 'services']] = 0
category_table_test = category_table_test.fillna('None')
category_table_test = category_table_test.apply(pp_pc, axis=1)

In [20]:
# pd.set_option('display.max_rows', 10)
category_table_train[category_table_train.iloc[:,1:].sum(axis=1)==0]['product_category'].value_counts()

product_category
                                              17489
control                                         263
standalone                                       10
allinone_rmk                                      7
Category                                          1
standard                                          1
retaildigital                                     1
Channel Partner done FTS Details in System        1
آخر                                               1
Name: count, dtype: int64

In [21]:
df_train = pd.concat([df_train, category_table_train.iloc[:,1:]], axis=1)
df_test = pd.concat([df_test, category_table_test.iloc[:,1:]], axis=1)

####  customer_position

In [22]:
# 17) customer_position

def pp_cp(text):
    text = text.replace('/', ' ').lower()

    position_map = {
        'executive': ['ceo', 'executive', 'president', 'entrepre', 'found', 'gerente', 'proprie', 'genel', 'chairman', 'boss', 'vp'],
        'trainee': ['intern', 'unpaid'],
        'vice president': ['vicepresident'],
        'engineering': ['installer', 'técnico', 'tech', 'desarrollo'],
        'lecturer': ['profess', 'lecture', 'educa', 'teacher', 'principal', 'academic', 'prof', 'faculty', 'hon dean', 'exam', 'pgt'],
        'none': ['bulgaria', 'not applicable'],
        'manufacturer': ['manufacturer'],
        'customer': ['user'],
        'entry level': ['entry'],
        'consultant': ['consult', 'career coach'],
        'decision maker': ['decision'],
        'director': ['director'],
        'hospital': ['főorvos', 'tierarzt', 'medical'],
        'exhibition': ['exhibition'],
        'employee': ['this is', 'mindenes', 'no influence'],
        'sales': ['sales'],
        'research': ['research', 'pathologist'],
        'others': ['other'],
        'partner': ['distributor', 'partner'],
        'management': ['manage']
    }

    for position, keywords in position_map.items():
        for keyword in keywords:
            if keyword in text:
                return position

    return text

df_train['customer_position'] = df_train['customer_position'].apply(pp_cp)
df_test['customer_position'] = df_test['customer_position'].apply(pp_cp)

#### expected_timeline

In [23]:
# expected_timeline

def pp_et(text):
    text = str(text).replace('_', ' ').replace('.','').replace(' - ', ' ~ ')
    if text not in ['nan', 'less than 3 months', '3 months ~ 6 months', 'more than a year','9 months ~ 1 year', '6 months ~ 9 months', 'less than 6 months', 'etc','being followed up']:
        return 'memo'
    return text

df_train['expected_timeline'] = pd.DataFrame(df_train['expected_timeline'].apply(pp_et))
df_test['expected_timeline'] = pd.DataFrame(df_test['expected_timeline'].apply(pp_et))

## 3. 데이터 전처리

### 1) 레이블 인코딩

In [24]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [25]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "bant_submit",
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
    


다시 학습 데이터와 제출 데이터를 분리합니다.

In [26]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [27]:
df_train = df_train.drop(['customer_country.1'], axis = 1)
df_test = df_test.drop(['customer_country.1'], axis = 1)

In [28]:
df_train = df_train.drop(['product_category', 'product_subcategory', 'product_modelname'], axis = 1)
df_test = df_test.drop(['product_category', 'product_subcategory', 'product_modelname'], axis = 1)

In [29]:
df_train.columns

Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'customer_position', 'response_corporate',
       'expected_timeline', 'ver_cus', 'ver_pro', 'ver_win_rate_x',
       'ver_win_ratio_per_bu', 'business_area', 'business_subarea',
       'lead_owner', 'is_converted', 'display', 'tv', 'temperature', 'air',
       'beam', 'beauty', 'board', 'cloud', 'pc', 'software', 'refrigerator',
       'robot', 'energy', 'other', 'parts_and_accessories', 'plug',
       'appliances', 'hospital', 'services'],
      dtype='object')

### 2) 학습, 검증 데이터 분리

In [31]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

## 4. 모델링

### 1) 모델 정의 

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
     

In [33]:
numeric_features = ['com_reg_ver_win_rate', 'historical_existing_cnt', 'lead_desc_length', 'ver_win_rate_x', 'ver_win_ratio_per_bu']
categorical_features = ['bant_submit', 'customer_country', 'business_unit', 'customer_idx', 'customer_type', 'enterprise', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job',
                        'inquiry_type', 'customer_position', 'response_corporate', 'expected_timeline', 'ver_cus',
                        'ver_pro', 'business_area', 'business_subarea', 'lead_owner', 'display', 'tv', 'temperature', 'air', 'beam', 'beauty',
                        'board', 'cloud', 'pc', 'software', 'refrigerator', 'robot', 'energy',
                        'other', 'parts_and_accessories', 'plug', 'appliances', 'hospital', 'services']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [36]:
model = CatBoostClassifier(random_state=400, verbose=0)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

In [37]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['com_reg_ver_win_rate',
                                                   'historical_existing_cnt',
                                                   'lead_desc_length',
                                                   'ver_win_rate_x',
                                                   'ver_win_ratio_per_bu']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEnc...
                                                   'idit_strategic_ver',
                                                   'customer_job',
                                                   'inquiry_type',
                                                   'customer_position',
                                                   'response_corporate',
                                                   'expected_timeline',
                                                   'ver_cus', 'ver_pro',
                                                   'business_area',
                                                   'business_subarea',
                                                   'lead_owner', 'display',
                                                   'tv', 'temperature', 'air',
                                                   'beam', 'beauty', 'board',
                                                   'cloud', 'pc', 'software',
                                                   'refrigerator', ...])])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7f1c4ef5cdc0>)])

In [38]:
y_val_pred = pipeline.predict(x_val)

In [39]:
y_val_pred = (y_val_pred == 'True')

# print("Classification Report:\n", classification_report(y_val, y_val_pred))
# print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
# print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
# print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
# print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
# print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

In [40]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

# pred = (model.predict(x_val.fillna(0)) == True)  # 문자열 레이블을 불리언으로 변환
get_clf_eval(y_val, y_val_pred)


오차행렬:
 [[  635   312]
 [   37 10876]]

정확도: 0.9706
정밀도: 0.9449
재현율: 0.6705
F1: 0.7844


-----------------------------------------------

- 정밀도와 재현율은 트레이드오프 관계

## 4. 제출하기

### 1) 테스트 데이터 예측

In [41]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [42]:
test_pred = pipeline.predict(x_test.fillna(0))
sum(test_pred == 'True') # True로 예측된 개수


411

### 2) 제출 파일 작성

In [43]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)